In [78]:
import tensorflow as tf
import pandas as pd
import lime
import lime.lime_tabular
import numpy as np

In [33]:
saved_filepath = "models/nn_model"
nn_model = tf.keras.models.load_model(saved_filepath)
df = pd.read_csv("./dataset/users_df_dataset_cleaned_with_indicators.csv")

In [34]:
df = df[[
#    "user_id"  ,
#    "name"  ,
#    "lang"  ,
    "bot"  ,
#    "created_at" ,
    "statuses_count" ,
    "account_age_in_days" ,
    "number_of_tweets" ,
    "account_average_tweets_per_day" ,
    "avg_tweets_per_actual_day" ,
#    "day_with_most_tweets" ,
    "max_number_of_tweets_in_a_day",
    "entropy_for_day",
    "entropy_for_hour",
    "entropy_for_minute",
    "avg_hashtags",
    "avg_text_length",
    "avg_mentions",
    "avg_special_char_in_text",
    "total_likes",
    "avt_favorite_count",
    "total_replies",
    "avt_reply_count",
    "total_retweet_count",
    "account_discussion_creation_ratio",
    "tweet_num_likes_ratio",
    "tweet_num_replies_ratio",
    "entropy_original_text",
    "entropy_text",
    "mean_inactive_period_length_in_seconds",
    "median_inactive_period_length_in_seconds",
    "mode_inactive_period_length_in_seconds",
    "mode_count"
]]

df = df.sample(frac=1, random_state=1).reset_index() #the same random state of the model 
df.pop("bot")

tr_size = 0.7 # the same for the model

tr_index = round(df.shape[0] * tr_size)

df_train = df[0:tr_index]


In [73]:
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=df_train.values, 
    mode="classification",
    verbose=True,
    class_names=("human", "bot"),
    sample_around_instance=True , # WARNING don't know if it is good
    feature_names = df.columns.tolist()
    )

In [96]:
instance_index = 1
exp = lime_explainer.explain_instance(
    data_row = df.values[instance_index], 
    predict_fn =  lambda x: np.array([nn_model.predict(x), 1 - nn_model.predict(x)]).reshape(1, -1), 
    top_labels=2
   )

exp.show_in_notebook(
    show_predicted_value=True,
    show_table=True,
    show_all=True)

157/157 [==============================] - 0s 822us/step


/home/matteo/.local/lib/python3.10/site-packages/lime/lime_tabular.py:372: UserWarning: 
                    Prediction probabilties do not sum to 1, and
                    thus does not constitute a probability space.
                    Check that you classifier outputs probabilities
                    (Not log probabilities, or actual class predictions).
                    
  warnings.warn("""


ValueError: Found input variables with inconsistent numbers of samples: [5000, 1]

In [97]:
x = df.values[instance_index]
np.array([nn_model.predict(x), 1 - nn_model.predict(x)]).reshape(1, -1)

1/1 [==============================] - 0s 16ms/step


array([[0.04027544, 0.95972455]], dtype=float32)